### Word Group In Youtube

In [153]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [154]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [155]:
# language pair
lang_folder = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 20000  # native word end index
unique_word_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 7  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 35  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [156]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [157]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [158]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [159]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [160]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [161]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [162]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
...,...,...
420724,snowflakes,5
420725,cazzone,5
420726,tonmawr,5
420727,halstedter,5


In [163]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [164]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,que,37853284
1,de,37809537
2,no,33043466
3,a,25439588
4,la,24024343
...,...,...
9995,amigable,6094
9996,abel,6092
9997,want,6091
9998,zombi,6090


In [165]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [166]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [167]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

9997

#### Youtube Sentence Analysis

In [168]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:07.500,00:00:10.780,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8
1,00:00:10.780,00:00:14.860,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8
2,00:00:14.860,00:00:18.900,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8
3,00:00:18.900,00:00:23.400,con más monumentos por metro cuadrado palacios...,AgBS7sIM6I8
4,00:00:23.400,00:00:26.140,calles estrechas y construcciones modernas con...,AgBS7sIM6I8
...,...,...,...,...
1303350,00:12:55.540,00:13:00.839,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU
1303351,00:13:00.839,00:13:01.850,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU
1303352,00:13:01.850,00:13:06.379,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU
1303353,00:13:06.379,00:13:24.050,una vez más y hasta,kKeXdLAy5tU


In [169]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [170]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,7.500,10.780,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8
1,1,10.780,14.860,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8
2,2,14.860,18.900,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8
3,3,18.900,23.400,con más monumentos por metro cuadrado palacios...,AgBS7sIM6I8
4,4,23.400,26.140,calles estrechas y construcciones modernas con...,AgBS7sIM6I8
...,...,...,...,...,...
1303350,1303350,775.540,780.839,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU
1303351,1303351,780.839,781.850,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU
1303352,1303352,781.850,786.379,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU
1303353,1303353,786.379,804.050,una vez más y hasta,kKeXdLAy5tU


In [171]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [172]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [173]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,7.500,10.780,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8,53.8,"cantado, sevilla","y, se, hablado, escrito, muchísimo, de, ha"
1,1,10.780,14.860,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8,76.9,"flamenco, importantísima","para, la, muchos, y, del, españa, de, cuna, hi..."
2,2,14.860,18.900,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8,87.5,,"es, más, uno, del, lugares, los, ciudades, est..."
3,3,18.900,23.400,con más monumentos por metro cuadrado palacios...,AgBS7sIM6I8,50.0,"cuadrado, palacios, patios, monumentos, monast...","árabes, más, metro, por, bosques, con"
4,4,23.400,26.140,calles estrechas y construcciones modernas con...,AgBS7sIM6I8,33.3,"estrechas, conviven, construcciones, modernas","calles, y"
...,...,...,...,...,...,...,...,...
1303350,1303350,775.540,780.839,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU,64.7,"volverlos, directos, liveperactina","siempre, en, no, si, y, podéis, los, a, temas,..."
1303351,1303351,780.839,781.850,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU,100.0,,"nombre, gracias, la, seguidores, idea, los, me..."
1303352,1303352,781.850,786.379,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU,100.0,,"espero, mucho, más, gracias, y, muchas, por, e..."
1303353,1303353,786.379,804.050,una vez más y hasta,kKeXdLAy5tU,100.0,,"más, vez, y, hasta, una"


In [174]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,7.500,10.780,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8,53.8,"cantado, sevilla","y, se, hablado, escrito, muchísimo, de, ha"
1,1,10.780,14.860,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8,76.9,"flamenco, importantísima","para, la, muchos, y, del, españa, de, cuna, hi..."
2,2,14.860,18.900,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8,87.5,NaN,"es, más, uno, del, lugares, los, ciudades, est..."
3,3,18.900,23.400,con más monumentos por metro cuadrado palacios...,AgBS7sIM6I8,50.0,"cuadrado, palacios, patios, monumentos, monast...","árabes, más, metro, por, bosques, con"
4,4,23.400,26.140,calles estrechas y construcciones modernas con...,AgBS7sIM6I8,33.3,"estrechas, conviven, construcciones, modernas","calles, y"
...,...,...,...,...,...,...,...,...
1303350,1303350,775.540,780.839,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU,64.7,"volverlos, directos, liveperactina","siempre, en, no, si, y, podéis, los, a, temas,..."
1303351,1303351,780.839,781.850,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU,100.0,NaN,"nombre, gracias, la, seguidores, idea, los, me..."
1303352,1303352,781.850,786.379,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU,100.0,NaN,"espero, mucho, más, gracias, y, muchas, por, e..."
1303353,1303353,786.379,804.050,una vez más y hasta,kKeXdLAy5tU,100.0,NaN,"más, vez, y, hasta, una"


In [175]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,7.500,10.780,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8,53.8,"cantado, sevilla","y, se, hablado, escrito, muchísimo, de, ha"
1,1,10.780,14.860,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8,76.9,"flamenco, importantísima","para, la, muchos, y, del, españa, de, cuna, hi..."
2,2,14.860,18.900,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8,87.5,NaN,"es, más, uno, del, lugares, los, ciudades, est..."
3,3,18.900,23.400,con más monumentos por metro cuadrado palacios...,AgBS7sIM6I8,50.0,"cuadrado, palacios, patios, monumentos, monast...","árabes, más, metro, por, bosques, con"
4,4,23.400,26.140,calles estrechas y construcciones modernas con...,AgBS7sIM6I8,33.3,"estrechas, conviven, construcciones, modernas","calles, y"
...,...,...,...,...,...,...,...,...
1254237,1303350,775.540,780.839,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU,64.7,"volverlos, directos, liveperactina","siempre, en, no, si, y, podéis, los, a, temas,..."
1254238,1303351,780.839,781.850,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU,100.0,NaN,"nombre, gracias, la, seguidores, idea, los, me..."
1254239,1303352,781.850,786.379,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU,100.0,NaN,"espero, mucho, más, gracias, y, muchas, por, e..."
1254240,1303353,786.379,804.050,una vez más y hasta,kKeXdLAy5tU,100.0,NaN,"más, vez, y, hasta, una"


In [176]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,0,7.500,10.780,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8,53.8,"cantado, sevilla","y, se, hablado, escrito, muchísimo, de, ha",se ha escrito se ha hablado y se ha
1,1,10.780,14.860,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8,76.9,"flamenco, importantísima","para, la, muchos, y, del, españa, de, cuna, hi...",de la historia de españa y para muchos
2,2,14.860,18.900,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8,87.5,NaN,"es, más, uno, del, lugares, los, ciudades, est...",una de las ciudades más bonitas del mundo este...
5,5,26.140,29.500,creando una de las ciudades más reconocibles q...,AgBS7sIM6I8,90.9,reconocibles,"más, se, ciudades, que, las, creando, de, una,...",creando una de las ciudades más
6,6,35.400,39.540,en este vídeo os voy a pedir que os sentéis y ...,AgBS7sIM6I8,80.0,"veáis, sentéis","pedir, en, voy, el, y, final, este, a, os, que...",en este vídeo os voy a pedir que os
...,...,...,...,...,...,...,...,...,...
1254236,1303349,770.569,775.540,y me tenéis como siempre en los directos de tw...,kKeXdLAy5tU,76.5,"twitch, directos","siempre, más, en, tenéis, como, profundidad, y...",y me tenéis como siempre en los
1254237,1303350,775.540,780.839,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU,64.7,"volverlos, directos, liveperactina","siempre, en, no, si, y, podéis, los, a, temas,...",los temas y si no podéis asistir a los
1254238,1303351,780.839,781.850,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU,100.0,NaN,"nombre, gracias, la, seguidores, idea, los, me...",gracias a los seguidores que me dieron la mara...
1254239,1303352,781.850,786.379,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU,100.0,NaN,"espero, mucho, más, gracias, y, muchas, por, e...",y nada más espero que os haya gustado mucho es...


In [177]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,se ha escrito se ha hablado y se ha,8.012500,9.908750,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8
1,de la historia de españa y para muchos,12.793506,14.860000,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8
2,una de las ciudades más bonitas del mundo este...,14.860000,18.900000,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8
3,creando una de las ciudades más,26.140000,27.769091,creando una de las ciudades más reconocibles q...,AgBS7sIM6I8
4,en este vídeo os voy a pedir que os,35.400000,37.658182,en este vídeo os voy a pedir que os sentéis y ...,AgBS7sIM6I8
...,...,...,...,...,...
915935,y me tenéis como siempre en los,770.569000,772.336467,y me tenéis como siempre en los directos de tw...,kKeXdLAy5tU
915936,los temas y si no podéis asistir a los,775.540000,777.715379,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU
915937,gracias a los seguidores que me dieron la mara...,780.839000,781.850000,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU
915938,y nada más espero que os haya gustado mucho es...,781.850000,786.379000,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU


In [178]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,se ha escrito se ha hablado y se ha,7.712500,10.208750,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8
1,de la historia de españa y para muchos,12.493506,15.160000,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8
2,una de las ciudades más bonitas del mundo este...,14.560000,19.200000,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8
3,creando una de las ciudades más,25.840000,28.069091,creando una de las ciudades más reconocibles q...,AgBS7sIM6I8
4,en este vídeo os voy a pedir que os,35.100000,37.958182,en este vídeo os voy a pedir que os sentéis y ...,AgBS7sIM6I8
...,...,...,...,...,...
915935,y me tenéis como siempre en los,770.269000,772.636467,y me tenéis como siempre en los directos de tw...,kKeXdLAy5tU
915936,los temas y si no podéis asistir a los,775.240000,778.015379,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU
915937,gracias a los seguidores que me dieron la mara...,780.539000,782.150000,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU
915938,y nada más espero que os haya gustado mucho es...,781.550000,786.679000,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU


In [179]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,se ha escrito se ha hablado y se ha,8,10,de sevilla se ha escrito se ha hablado y se ha...,AgBS7sIM6I8
1,de la historia de españa y para muchos,12,15,cuna del flamenco parte importantísima de la h...,AgBS7sIM6I8
2,una de las ciudades más bonitas del mundo este...,15,19,una de las ciudades más bonitas del mundo este...,AgBS7sIM6I8
3,creando una de las ciudades más,26,28,creando una de las ciudades más reconocibles q...,AgBS7sIM6I8
4,en este vídeo os voy a pedir que os,35,38,en este vídeo os voy a pedir que os sentéis y ...,AgBS7sIM6I8
...,...,...,...,...,...
915935,y me tenéis como siempre en los,770,773,y me tenéis como siempre en los directos de tw...,kKeXdLAy5tU
915936,los temas y si no podéis asistir a los,775,778,los temas y si no podéis asistir a los directo...,kKeXdLAy5tU
915937,gracias a los seguidores que me dieron la mara...,781,782,gracias a los seguidores que me dieron la mara...,kKeXdLAy5tU
915938,y nada más espero que os haya gustado mucho es...,782,787,y nada más espero que os haya gustado mucho es...,kKeXdLAy5tU


In [180]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,del mal amén gloria al padre gloria al hijo y ...,1,2,oración contra todos los peligros de san cipri...,Uu4s5lZ4pSI
1,debería haber ido un poco más lento y haber un...,702,720,debería haber ido un poco más lento y haber un...,LTKnQFlBP9M
2,puedes intentar moverte desde el exterior esto...,287,303,puedes intentar moverte desde el exterior esto...,oCSJ5tZio4k
3,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8
4,agarre un poco de experiencia en mi proyecto p...,574,590,agarre un poco de experiencia en mi proyecto p...,NCwnlTkMdLU
...,...,...,...,...,...
915935,c a r r e,421,426,c a r r e,GxUyC_hL1L4
915936,l m n o p,60,63,l m n o p,iPe7dHumRfQ
915937,l m n o p,101,104,l m n o p,iPe7dHumRfQ
915938,a e i o u,525,529,hummm a e i o u,RV3m4kHjqZc


In [181]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,del mal amén gloria al padre gloria al hijo y ...,1,2,oración contra todos los peligros de san cipri...,Uu4s5lZ4pSI,41
1,debería haber ido un poco más lento y haber un...,702,720,debería haber ido un poco más lento y haber un...,LTKnQFlBP9M,75
2,puedes intentar moverte desde el exterior esto...,287,303,puedes intentar moverte desde el exterior esto...,oCSJ5tZio4k,74
3,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8,87
4,agarre un poco de experiencia en mi proyecto p...,574,590,agarre un poco de experiencia en mi proyecto p...,NCwnlTkMdLU,83
...,...,...,...,...,...,...
915935,c a r r e,421,426,c a r r e,GxUyC_hL1L4,80
915936,l m n o p,60,63,l m n o p,iPe7dHumRfQ,100
915937,l m n o p,101,104,l m n o p,iPe7dHumRfQ,100
915938,a e i o u,525,529,hummm a e i o u,RV3m4kHjqZc,100


In [182]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8
1,sin embargo lo que aún no se ha resuelto por c...,565,576,sin embargo lo que aún no se ha resuelto por c...,QPzFLbJVnaE
2,justamente debajo y para más casualidad termin...,116,127,y ya de paso hay unos toboganes justamente deb...,xqr2skItS3E
3,pero sí es cierto que si hubiese cables por en...,461,471,pero sí es cierto que si hubiese cables por en...,zTxdXfa8zlA
4,fuera de control desde la izquierda por lo tan...,240,250,y dos vettel viene vagamente fuera de control ...,4WuN3Jike84
...,...,...,...,...,...
767470,no a b o c,453,456,no a b o c,_iH2XsJaGyk
767471,l m n o p,35,38,l m n o p,iPe7dHumRfQ
767472,l m n o p,60,63,l m n o p,iPe7dHumRfQ
767473,l m n o p,101,104,l m n o p,iPe7dHumRfQ


In [183]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-183-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8,92
1,sin embargo lo que aún no se ha resuelto por c...,565,576,sin embargo lo que aún no se ha resuelto por c...,QPzFLbJVnaE,66
2,justamente debajo y para más casualidad termin...,116,127,y ya de paso hay unos toboganes justamente deb...,xqr2skItS3E,86
3,pero sí es cierto que si hubiese cables por en...,461,471,pero sí es cierto que si hubiese cables por en...,zTxdXfa8zlA,100
4,fuera de control desde la izquierda por lo tan...,240,250,y dos vettel viene vagamente fuera de control ...,4WuN3Jike84,79
...,...,...,...,...,...,...
767470,no a b o c,453,456,no a b o c,_iH2XsJaGyk,100
767471,l m n o p,35,38,l m n o p,iPe7dHumRfQ,100
767472,l m n o p,60,63,l m n o p,iPe7dHumRfQ,100
767473,l m n o p,101,104,l m n o p,iPe7dHumRfQ,100


In [184]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8
1,justamente debajo y para más casualidad termin...,116,127,y ya de paso hay unos toboganes justamente deb...,xqr2skItS3E
2,pero sí es cierto que si hubiese cables por en...,461,471,pero sí es cierto que si hubiese cables por en...,zTxdXfa8zlA
3,qué pasa el chaval jamás hasta ese momento ha ...,84,93,qué pasa el chaval jamás hasta ese momento ha ...,3dc1Dwjlwys
4,desde luego no es mala idea porque él no está ...,425,433,desde luego no es mala idea porque él no está ...,Jw5JFE_7icI
...,...,...,...,...,...
231327,y e o como h,44,45,y e o como h,Q_87VRj7GB0
231328,no a b o c,453,456,no a b o c,_iH2XsJaGyk
231329,l m n o p,35,38,l m n o p,iPe7dHumRfQ
231330,l m n o p,60,63,l m n o p,iPe7dHumRfQ


In [185]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,pasa bien hay menos mala habitación,zz98Q9ce5Wc
1,mucho más si es tanto mejor vivimos con ello l...,zyHrpu_0GWI
2,totalmente gratis podéis activar también,zuF48G1QkQ8
3,tan alto soy tu maestro ahora,zoXpPyjA3zI
4,gente vive así ahora vivimos pues ha pasado na...,zn4B5zDWOGk
...,...,...
2067,bastante interesante tiene ofrece algo diferen...,06Z051nXubs
2068,ricos comida ya distancia industria ya fue muy...,04_MJ0qsZ98
2069,estas muchas ganas mucha carne dentro agua,04_MJ0qsZ98
2070,aprender representar participar completar coop...,03nct36UcZw


In [186]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8
1,justamente debajo y para más casualidad termin...,116,127,y ya de paso hay unos toboganes justamente deb...,xqr2skItS3E
2,pero sí es cierto que si hubiese cables por en...,461,471,pero sí es cierto que si hubiese cables por en...,zTxdXfa8zlA
3,qué pasa el chaval jamás hasta ese momento ha ...,84,93,qué pasa el chaval jamás hasta ese momento ha ...,3dc1Dwjlwys
4,desde luego no es mala idea porque él no está ...,425,433,desde luego no es mala idea porque él no está ...,Jw5JFE_7icI
...,...,...,...,...,...
1994,i hope you like it,200,201,i hope you like it,rWqNMm3s3N8
1995,you know how we do,104,106,you know how we do,ViIWwDNp9XM
1996,cha chan i got it,1633,1635,cha chan i got it,9Vl8rty5hqs
1997,you know who i am,34,36,you know who i am,qvFFZViAFkg


In [187]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,básicamente los autos hoy no pueden competir e...,46,67,básicamente los autos hoy no pueden competir e...,ZMOu81BhyH8,https://www.youtube.com/watch?v=ZMOu81BhyH8&t=46s
1,justamente debajo y para más casualidad termin...,116,127,y ya de paso hay unos toboganes justamente deb...,xqr2skItS3E,https://www.youtube.com/watch?v=xqr2skItS3E&t=...
2,pero sí es cierto que si hubiese cables por en...,461,471,pero sí es cierto que si hubiese cables por en...,zTxdXfa8zlA,https://www.youtube.com/watch?v=zTxdXfa8zlA&t=...
3,qué pasa el chaval jamás hasta ese momento ha ...,84,93,qué pasa el chaval jamás hasta ese momento ha ...,3dc1Dwjlwys,https://www.youtube.com/watch?v=3dc1Dwjlwys&t=84s
4,desde luego no es mala idea porque él no está ...,425,433,desde luego no es mala idea porque él no está ...,Jw5JFE_7icI,https://www.youtube.com/watch?v=Jw5JFE_7icI&t=...
...,...,...,...,...,...,...
1994,i hope you like it,200,201,i hope you like it,rWqNMm3s3N8,https://www.youtube.com/watch?v=rWqNMm3s3N8&t=...
1995,you know how we do,104,106,you know how we do,ViIWwDNp9XM,https://www.youtube.com/watch?v=ViIWwDNp9XM&t=...
1996,cha chan i got it,1633,1635,cha chan i got it,9Vl8rty5hqs,https://www.youtube.com/watch?v=9Vl8rty5hqs&t=...
1997,you know who i am,34,36,you know who i am,qvFFZViAFkg,https://www.youtube.com/watch?v=qvFFZViAFkg&t=34s


In [188]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [189]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [190]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [191]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Spanish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'Spanish_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx']

In [192]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [193]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# # shared word frequency
# shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000